# PyLadies Ecuador Workshop 
# Pandas para Ciencia de Datos @ Junio 2019 

Andrea Martínez E.

## Resumen

En este taller se hará una breve introducción al manejo de datos con Python utilizando la librería Pandas en un notebook de Jupyter.

De manera general se tratarán los siguientes temas:

    1) Carga e inspección inicial de los datos
    2) Limpieza de datos
    3) Análisis de datos
    
Para esto se va a utilizar información estadística del INEC referente a los registros de nacidos vivos del año 2017 que se encuentra en el link:

http://www.ecuadorencifras.gob.ec/nacimientos_y_defunciones/

### Importar librerías

Es importante, al inicio, importar las librerías que vamos a utilizar.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

## 1) Carga e inspección inicial de los datos

Para cargar la información en el entorno de trabajo se utiliza la función de Pandas `read_csv` que almacena los datos desde un archivo delimitado por comas (CSV) a un `DataFrame`. Todos los parametros para importar un archivo csv en pandas se pueden consultar en el link: https://honingds.com/blog/pandas-read_csv/

<span style="color:blue">**Se debe parametrizar a `encoding=latin1` ya que caso contrario nos da un error.**</span>

In [ ]:
df = pd.read_csv('ENV_2017.csv',encoding='latin1')

<span style="color:blue">**Se obtiene una advertencia que ciertas columnas tiene tipos de datos mezclados, por eso vamos a exportar todos los datos como tipo `string` y guardar en una lista las columnas con problemas para revisar en el paso 2.**</span> 

In [ ]:
df = pd.read_csv('ENV_2017.csv',encoding='latin1',dtype=str)
col_revisar=[3,5,12,13,14,16,17,27,29,31,32,36,37]

**¿Qué es un DataFrame?** 
Es una collección ordenada de filas y columnas como una hoja de excel o una base de datos en SQL. Tiene un index para filas y  columnas. Cada columna puede tener datos de diversos tipos.

A continuación vamos a revisar ciertos atributos del dataframe.

In [ ]:
# Para saber el número de filas y columnas
df.shape

In [ ]:
# Para el número de filas se puede utilizar el primer argumento de la función shape
df.shape[0]

In [ ]:
# O también
len(df)

In [ ]:
# Para el número de columnas se puede utilizar el segundo argumento de la función shape
df.shape[1]

In [ ]:
#Para mirar nuestro dataframe unicamente escribimos su nombre
df

In [ ]:
#Como el dataframe tiene muchas filas, podemos desplegar las 5 primeras
df.head()

In [ ]:
# Para que se presenten todas las columnas, escribimos
pd.options.display.max_columns = None
df.head(10)

In [ ]:
# Podemos visualizar una muestra aleatoria de los datos
df.sample(20)

In [ ]:
# Para revisar los nombres de las columnas
df.columns

In [ ]:
# Para revisar los tipos de datos de cada columna (en este caso todas son string 'object')
df.dtypes

## Ejercicio
Mostrar en la siguiente celda una muestra aleatoria de 15 filas del dataframe "df"

## Acceder a las columnas
Vamos a revisar distintos comandos para acceder a las columnas en un dataframe

In [ ]:
df.prov_nac.head()

In [ ]:
#Tambien se puede usar cuando los nombres de las columnas tengan espacios o caracteres especiales
df['prov_nac'].head()

In [ ]:
#Varias columnas
df[['prov_nac','cant_nac','parr_nac']].head()

In [ ]:
#Se puede usar el comando loc 
df.loc[:,'prov_nac'].head()

In [ ]:
#Se puede usar el comando loc en varias columnas
df.loc[:,['prov_nac','cant_nac','parr_nac']].head()

In [ ]:
#Se puede usar el comando iloc para acceder a las columnas con la posición
df.iloc[:,3].head()

In [ ]:
#Se puede usar iloc con varias columnas
df.iloc[:,col_revisar].sample(20)

In [ ]:
#Se puede acceder a los valores únicos de una columna
df.peso.unique()

In [ ]:
df.apgar5.unique()

In [ ]:
#Se puede contar cuantos valores únicos 
df.anio_nac.nunique()

## Ejercicios
Mostrar los valores únicos de la columna "cant_res" del dataframe "df"

Mostrar los primeros 10 valores de la columna 'cod_pais'

## Acceder a las filas

In [ ]:
#Se utiliza el comando loc
df.loc[df['parr_nac'] == 'Conocoto']

In [ ]:
#Varias condiciones
df.loc[
    (df['parr_nac'] == 'Conocoto') 
    & (df['sem_gest'] == '40')
]

## 2) Limpieza de datos

Lo primero que debemos corregir es la información de las columnas con problemas

In [ ]:
#Para obtener los nombres de las columnas con problemas
col_names_rev=df.columns[col_revisar]
col_names_rev

In [ ]:
#Revisamos los valores unicos
df.dia_insc.unique()

In [ ]:
#Vemos que son columnas con información numérica
for i in col_revisar:
    df.iloc[:, i] = df.iloc[:, i].str.replace('Sin información', '').str.replace(' ', '')

In [ ]:
#Revisamos nuevamente los valores unicos
df.dia_insc.unique()

In [ ]:
#Transformamos estas columnas a números enteros
df[col_names_rev] = df[col_names_rev].apply(pd.to_numeric)

In [ ]:
#Revisamos nuevamente los valores unicos
df.dia_insc.unique()

In [ ]:
#Revisamos los tipos de datos en cada columna
df.dtypes

In [ ]:
#Falta transformar algunas columnas a enteros y fechas
col_int = ['anio_nac','dia_nac','num_emb','num_par','hij_viv'] #Columnas tipo entero
col_fecha  = ['fecha_nac'] #Columnas tipo fecha

In [ ]:
#Columnas tipo entero
for col in col_int:
    df[col] = df[col].astype(dtype=np.int64)

In [ ]:
#Columnas tipo fecha
df[col_fecha] = df[col_fecha].apply(pd.to_datetime)

In [ ]:
#Creamos una columna para identificar cada registro
df['registroID'] = df.index
df.registroID.head()

In [ ]:
#Contamos los valores nulos
df.isnull().sum().sort_values(ascending=False)

<span style="color:blue">**El paso de limpieza de datos puede requerir métodos para completar la información faltante. En el caso de registros numéricos se pueden completar con la media o la mediana**</span>.

In [ ]:
#Definimos los valores con los que vamos a completar los valores nulos en cada columna
valores_na = {'anio_ins': 2017, 'mes_insc': 12, 'dia_insc': 31, 'talla': df.talla.mean(), 'peso': df.peso.mean(),
              'edad_mad': df.edad_mad.median()}
valores_na

In [ ]:
#Completamos los valores nulos
df=df.fillna(value=valores_na)

## 3) Análisis de datos
Vamos a analizar la información y se pueden plantear algunas preguntas.

**¿Cuántos registros de inscripción constan en la base de datos?**

In [ ]:
print('Cantidad de registros de la base de datos:', len(df))

**¿Cuántos nacidos vivos fueron inscritos en el año 2017?**

In [ ]:
print('Cantidad de nacidos vivos inscritos en el año 2017:', df.loc[df.anio_insc == 2017].registroID.nunique())

**¿Cuál es el numero de nacidos vivos por edad de la madre?**

In [ ]:
df.groupby('edad_mad').count()[['registroID']]

In [ ]:
#Distribución gráfica de la variable edad
df.hist(column='edad_mad',bins=20)

**¿Cuál es el número de nacidos vivos por género?**

In [ ]:
df.groupby('sexo').count()[['registroID']]

In [ ]:
# En valores relativos
df.sexo.value_counts() / len(df)*100

**¿Cuál es el número de nacidos vivos prematuros (menos de 37 semanas de gestacion)?**

In [ ]:
# Se debe crear una nueva variable para indicar si es prematuro
df['es_prematuro']= np.where(df['sem_gest']<37,'Si','No')

In [ ]:
# En valores relativos
df.es_prematuro.value_counts() / len(df)*100

**¿Cuál es el número de nacidos vivos por rangos de edad de la madre para los nacimientos en el año 2017?**

In [ ]:
# Se debe crear una nueva variable para indicar el rango de edad de la madre
df['rango_edad_mad']= np.where(df['edad_mad'] < 15, '10-14', 
                               (np.where(df['edad_mad'] > 19, 'Mayor a 20', '15-19')))

In [ ]:
# Podemos crear un nuevo dataframe con los valores de los nacidos en el 2017
df2 = df.loc[df.anio_nac==2017]

In [ ]:
# En terminos relativos solo 2017
df2.rango_edad_mad.value_counts() / len(df2)*1000

In [ ]:
#Se pueden realizar groupby con más de una variable
df.groupby(['rango_edad_mad','es_prematuro']).count()[['registroID']]

**¿Cuál es la parroquia con mayor número de nacimientos prematuros?**

In [ ]:
#Tambien se pueden ordenar los resultados
df.groupby(['prov_nac','cant_nac','parr_nac','es_prematuro']).count()[['registroID']].sort_values(by=['es_prematuro','registroID'],ascending=False)

In [ ]:
#Verificamos la información
df.describe()

## Ejercicios
Mostrar la distribución de las semanas de gestación, variable "sem_gest"

Crear una nueva variable para identificar si el nacido vivo tiene bajo peso (menos de 2500 gramos)